# 🍽️ iFood — Intelligent Segmentation and Campaign Response Analysis



## Overview

This project analyzes the **effectiveness of promotional campaigns in a premium retail context at iFood**, using an **adapted dataset** for analytical purposes. The scenario involves multiple campaigns with distinct strategies and a rich dataset containing **demographic, behavioral, and consumption** information.

The goal is to turn heterogeneous campaign results into **reliable business decisions**, supporting the evolution of a data-driven marketing strategy focused on performance, efficiency, and risk reduction.



## Project Objectives

- Evaluate **customer response** to different promotional strategies  
- Compare campaign performance through **controlled (A/B) tests**  
- Define criteria for **evidence-based decision making**  
- Support campaign planning with a focus on **impact and return on investment**



## Analytical Approach

The project includes the following steps:

- Exploration and preparation of customer data  
- Analysis of customer profiles, purchasing behavior, and campaign history  
- Comparison of test groups and performance evaluation across approaches  
- Definition of metrics, hypotheses, and decision criteria  
- Interpretation of results with a focus on **business implications**



## Data

The dataset includes variables such as **income, household composition, recency, spending by category, purchase channels, website visits, campaign acceptance history**, as well as demographic and customer relationship attributes. It also includes the identification of **test groups** for experimentation.


## Expected Outcome

To build a **clear and reproducible framework** for evaluating campaigns with analytical rigor and turning data into **actionable recommendations**, supporting marketing decisions focused on **performance, risk, and investment efficiency**.

---
